# Analysis Of Toronto

## Part#1

<p>Getting the librearies which will be using alongside the project</p>

In [1]:
import requests
import pandas as pd
import numpy as np

from geopy.geocoders import Nominatim
from IPython.display import Image 
from IPython.core.display import HTML 

import matplotlib.cm as cm
import matplotlib.colors as colors

import folium
from sklearn.cluster import KMeans

<p>Dataset import from Wikipedia</p>

In [2]:
dfRaw = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
dfData = dfRaw[0].iloc[:,:]
print("load Succesfully")

load Succesfully


<p>Cleaning the data and check that everything looks good to proceed.
</p>

In [3]:
print(dfData.shape)
dfData = dfData[~(dfData.Borough == "Not assigned")].reset_index(drop=True)
dfData.head(10)

(180, 3)


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


<p>Postal codes that apppear more than once</p>

In [4]:
uniq = dfData['Postal Code'].value_counts()
test = pd.DataFrame(uniq)
test[test['Postal Code'] > 1].head()

,Postal Code


<p>Neighborhood that contains the value "Not assigned"</p>

In [5]:
dfData[dfData.Neighbourhood == "Not assigned"].head()

,Postal Code,Borough,Neighbourhood


In [6]:
dfData.shape

(103, 3)

## Part#2

<p>Getting the longitude and longitude for the postal codes </p>

In [7]:
lldf = pd.read_csv("https://cocl.us/Geospatial_data")
lldf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<p>Merge the main data with longitude and latitude</p>

In [8]:
dfData = pd.merge(left=dfData,right=lldf,how="left",on="Postal Code")
dfData.head(12)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Part#3

Establishing the center point of the map of toronto

In [9]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
t_latitude = location.latitude
t_longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(t_latitude, t_longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


<p>Create map of Toronto using latitud and longitud values</p>

In [10]:
map_toronto = folium.Map(location=[t_latitude, t_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(dfData['Latitude'], dfData['Longitude'], dfData['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_toronto)    
map_toronto

New dataframe with only the Borough that contains the string "toronto"

In [11]:
dftoronto = dfData[dfData['Borough'].str.contains('(?i).*toronto.*')]
dftoronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


Map of Neighborhoods that Borough's name contains "toronto" in their name

In [12]:
map_toronto2 = folium.Map(location=[t_latitude, t_longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(dftoronto['Latitude'], dftoronto['Longitude'], dftoronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.9,
        parse_html=False
    ).add_to(map_toronto2)    
map_toronto2

Establishing credentials for Foursquare

In [13]:
code = 'RILA0BKQSO00ZAI3R4NOBF2QY40PEFLZSDY0OTPYSJUCQIMG'
CLIENT_ID = 'IDQPORJB14MMWJSIZZ1VOF4FQOJU5TKYIDRMD3U51YU4K0HC' # your Foursquare ID
CLIENT_SECRET = '5B2GKQ3DQPO4XTA133KZX2S434ITQ4JUWZFWQDE3QTK3UWZT' # your Foursquare Secret
ACCESS_TOKEN = 'S3DM0BPCF5KZJOCH3VQT5KNN2SVLFKPUJMVD3KRJECCYX2F0' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
#https://foursquare.com/oauth2/access_token?client_id=IDQPORJB14MMWJSIZZ1VOF4FQOJU5TKYIDRMD3U51YU4K0HC&client_secret=5B2GKQ3DQPO4XTA133KZX2S434ITQ4JUWZFWQDE3QTK3UWZT&grant_type=authorization_code&redirect_uri=https://www.google.com&code=RILA0BKQSO00ZAI3R4NOBF2QY40PEFLZSDY0OTPYSJUCQIMG
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IDQPORJB14MMWJSIZZ1VOF4FQOJU5TKYIDRMD3U51YU4K0HC
CLIENT_SECRET:5B2GKQ3DQPO4XTA133KZX2S434ITQ4JUWZFWQDE3QTK3UWZT


In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Establishing all the venues in Toronto

In [15]:
toronto_venues = getNearbyVenues(names=dftoronto['Neighbourhood'],
                                   latitudes=dftoronto['Latitude'],
                                   longitudes=dftoronto['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [16]:
print(toronto_venues.shape)
toronto_venues.head()

(863, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Checking the amount of Neighborhoods that contains Toronto as Borough

In [17]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",18,18,18,18,18,18
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,30,30,30,30,30,30
Christie,15,15,15,15,15,15
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,30,30,30,30,30,30


One hot encoding, relocating the column 'Neighborhood' into the start of the dataframe

In [18]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

col_names = toronto_onehot.columns.to_list()
col_names.remove("Neighborhood")
col_names = ["Neighborhood"] + col_names

toronto_onehot = toronto_onehot[col_names]

toronto_onehot.head()

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333
5,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.033333,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.066667,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [20]:
toronto_grouped.shape

(39, 190)

Getting the top 10 venues in our data

In [21]:
num_top_venues = 10

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0  Seafood Restaurant  0.07
1         Coffee Shop  0.07
2      Farmers Market  0.07
3            Beer Bar  0.07
4        Cocktail Bar  0.07
5         Fish Market  0.03
6            Creperie  0.03
7              Museum  0.03
8            Fountain  0.03
9   French Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
                   venue  freq
0                   Café  0.12
1  Performing Arts Venue  0.08
2            Coffee Shop  0.08
3              Nightclub  0.08
4         Breakfast Spot  0.08
5                    Gym  0.04
6     Italian Restaurant  0.04
7              Pet Store  0.04
8          Grocery Store  0.04
9           Climbing Gym  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0           Yoga Studio  0.06
1         Auto Workshop  0.06
2            Skate Park  0.06
3         Burrito Place  0.06
4    Light Rail Station  0.06
5

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Getting the top 15 venues for each Neighborhood into a dataframe

In [23]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Berczy Park,Seafood Restaurant,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Concert Hall,Tailor Shop,Bistro,Museum,Creperie,Basketball Stadium,Café,Bakery,Liquor Store,Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Nightclub,Coffee Shop,Performing Arts Venue,Breakfast Spot,Bakery,Climbing Gym,Convenience Store,Burrito Place,Restaurant,Intersection,Bar,Stadium,Pet Store,Italian Restaurant
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Spa,Gym / Fitness Center,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Park,Garden,Pizza Place,Butcher,Burrito Place,Recording Studio,Restaurant,Brewery
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boutique,Airport,Airport Food Court,Airport Gate,Boat or Ferry,Rental Car Location,Harbor / Marina,Sculpture Garden,Plane,Diner,Dessert Shop,Dog Run
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Yoga Studio,Poke Place,Spa,Seafood Restaurant,Sandwich Place,Ramen Restaurant,Bubble Tea Shop,Modern European Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Comic Shop,Japanese Restaurant


Creating the model for to compare N's

In [24]:
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

Adding the results of the model into the main dataframe

In [25]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = dftoronto

neighborhoods_venues_sorted.set_index('Neighborhood',inplace=True)
toronto_merged = pd.merge(left=toronto_merged,right=neighborhoods_venues_sorted,left_on='Neighbourhood',right_on='Neighborhood',how='left')

toronto_merged.head() 

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Café,Breakfast Spot,...,Bakery,Yoga Studio,Historic Site,Performing Arts Venue,Chocolate Shop,Pub,Restaurant,Dessert Shop,Spa,Event Space
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Yoga Studio,Diner,...,Beer Bar,College Auditorium,Italian Restaurant,Japanese Restaurant,Sandwich Place,Restaurant,Bank,Burger Joint,Burrito Place,Creperie
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Café,Coffee Shop,Theater,Music Venue,...,Hotel,Diner,Modern European Restaurant,Sandwich Place,Bookstore,Ramen Restaurant,Burrito Place,Mexican Restaurant,Electronics Store,Plaza
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Gastropub,Café,Coffee Shop,Farmers Market,...,Middle Eastern Restaurant,Jazz Club,Japanese Restaurant,Italian Restaurant,Creperie,Hotel,Cosmetics Shop,BBQ Joint,Restaurant,Gym
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Pub,Health Food Store,Yoga Studio,...,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cuban Restaurant,Creperie,Coworking Space,Convenience Store


Map of the Neighborhood's with the similiar tags by color

In [26]:
map_clusters = folium.Map(location=[t_latitude, t_longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster #1

In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
4,East Toronto,0,Trail,Pub,Health Food Store,Yoga Studio,Cosmetics Shop,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cuban Restaurant,Creperie,Coworking Space,Convenience Store


Cluster #2

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Park,Café,Breakfast Spot,Theater,Bakery,Yoga Studio,Historic Site,Performing Arts Venue,Chocolate Shop,Pub,Restaurant,Dessert Shop,Spa,Event Space
1,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,Yoga Studio,Diner,Smoothie Shop,Beer Bar,College Auditorium,Italian Restaurant,Japanese Restaurant,Sandwich Place,Restaurant,Bank,Burger Joint,Burrito Place,Creperie
2,Downtown Toronto,1,Café,Coffee Shop,Theater,Music Venue,Burger Joint,Hotel,Diner,Modern European Restaurant,Sandwich Place,Bookstore,Ramen Restaurant,Burrito Place,Mexican Restaurant,Electronics Store,Plaza
3,Downtown Toronto,1,Gastropub,Café,Coffee Shop,Farmers Market,New American Restaurant,Middle Eastern Restaurant,Jazz Club,Japanese Restaurant,Italian Restaurant,Creperie,Hotel,Cosmetics Shop,BBQ Joint,Restaurant,Gym
5,Downtown Toronto,1,Seafood Restaurant,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Concert Hall,Tailor Shop,Bistro,Museum,Creperie,Basketball Stadium,Café,Bakery,Liquor Store,Restaurant
6,Downtown Toronto,1,Coffee Shop,Italian Restaurant,Café,Yoga Studio,Poke Place,Spa,Seafood Restaurant,Sandwich Place,Ramen Restaurant,Bubble Tea Shop,Modern European Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Comic Shop,Japanese Restaurant
7,Downtown Toronto,1,Grocery Store,Café,Park,Restaurant,Coffee Shop,Nightclub,Italian Restaurant,Candy Store,Baby Store,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cuban Restaurant,Creperie
8,Downtown Toronto,1,Coffee Shop,Café,Seafood Restaurant,Restaurant,Asian Restaurant,Fast Food Restaurant,Speakeasy,Plaza,Gym / Fitness Center,Steakhouse,Concert Hall,Pizza Place,Sushi Restaurant,Smoke Shop,Thai Restaurant
9,West Toronto,1,Pharmacy,Bakery,Coffee Shop,Pool,Smoke Shop,Brewery,Bar,Bank,Supermarket,Café,Grocery Store,Music Venue,Park,Middle Eastern Restaurant,Comfort Food Restaurant
10,Downtown Toronto,1,Park,Hotel,Café,Plaza,Roof Deck,Skating Rink,Coffee Shop,Ice Cream Shop,Dessert Shop,Deli / Bodega,Salad Place,Dance Studio,Sporting Goods Shop,Japanese Restaurant,Bubble Tea Shop


Cluster #3

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
29,Central Toronto,2,Restaurant,Park,Cosmetics Shop,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cuban Restaurant,Creperie,Coworking Space,Yoga Studio,Cocktail Bar,Convenience Store


Cluster #4

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
18,Central Toronto,3,Park,Swim School,Bus Line,Yoga Studio,Coworking Space,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cuban Restaurant,Creperie,Cosmetics Shop,Dog Run
21,Central Toronto,3,Park,Trail,Sushi Restaurant,Jewelry Store,Yoga Studio,Coworking Space,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cuban Restaurant,Creperie,Convenience Store
33,Downtown Toronto,3,Park,Playground,Trail,Yoga Studio,Cosmetics Shop,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cuban Restaurant,Creperie,Coworking Space,Concert Hall
